In [ ]:
%matplotlib inline
import os
import shutil
import random
import torch
import torchvision
import numpy as np
import pandas as pd
from matplotlib import pyplot as plt
from sklearn.model_selection import train_test_split
from torch.utils.data import Dataset
from torch.utils.data import DataLoader
import torch.nn.functional as F
import torch.nn as nn

!pip install rasterio
import rasterio
import geopandas

import torchvision.transforms as transforms



     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.3/21.3 MB 76.8 MB/s eta 0:00:00


In [ ]:

def read_window(dataset, row, col, width=100, height=100):
    # Calculate the upper-left pixel coordinates of the window
    ul_col = max(0, col - width // 2)
    ul_row = max(0, row - height // 2)

    # Calculate the lower-right pixel coordinates of the window
    lr_col = min(dataset.shape[1], col + width // 2)
    lr_row = min(dataset.shape[0], row + height // 2)

     # Calculate the actual size of the window
    actual_width = lr_col - ul_col
    actual_height = lr_row - ul_row

    # Read the data within the window

    data = dataset[ul_row:lr_row, ul_col:lr_col]

    if actual_width < width or actual_height < height:
        pad_width = width - actual_width
        edge_width = pad_width // 2
        pad_height = height - actual_height
        edge_heigth = pad_height // 2
        data = np.pad(data, ((edge_heigth, pad_height-edge_heigth), (edge_width, pad_width-edge_width)), mode='mean')


    return data

In [ ]:
class extractWindows():
    def __init__(self, feature_readers, target_reader, add_feat):
        self.num_feat = len(feature_readers)
        self.feature_readers = feature_readers
        self.target_reader = target_reader
        self.add_feat = add_feat

    def __getFeatures__(self, row, col):
    #get for each feature the corresponding window "
        features = []
        for i in range(self.num_feat):
            data_feat = read_window(self.feature_readers[i], row, col)

            features.append(data_feat)

        features = np.array(features)

        point_feat = []
        window=((row, row+1), (col, col+1))
        for lay in self.add_feat:

            p_f = lay.read(1, window = window)


            point_feat.append(p_f[0])

        return features, np.array(point_feat)

    def __getTarget__(self, row, col):

        window=((row, row+1), (col, col+1))

        target = self.target_reader.read(1, window = window)

        return target




In [ ]:
from google.colab import drive
drive.mount('/content/drive/')
path_drive = r'drive/MyDrive/Landslides_data/'
#path = r'datasets/'

!unzip drive/MyDrive/Landslides_data/Features


Mounted at /content/drive/
Archive:  drive/MyDrive/Landslides_data/Features.zip
  inflating: aspect.tif              
  inflating: curvature.tif           
  inflating: dtm.tif                 
  inflating: geo_faults.tif          
  inflating: land_cover_raster.tif   
  inflating: prec_90.tif             
  inflating: prec_avr.tif            
  inflating: rivers.tif              
  inflating: roads.tif               
  inflating: slope_rad.tif           
  inflating: train_raster.tif        
  inflating: twi.tif                 


In [ ]:
path = ''
elevation = rasterio.open(path + 'dtm.tif')
elevation = elevation.read(1)
twi = rasterio.open(path + 'twi.tif')
twi = twi.read(1)
slope_rad = rasterio.open(path + 'slope_rad.tif')
slope_rad = slope_rad.read(1)
aspect = rasterio.open(path + 'aspect.tif')
aspect = aspect.read(1)
curvature = rasterio.open(path + 'curvature.tif')
curvature = curvature.read(1)

rain_90 = rasterio.open(path +'prec_90.tif')
rain_avr = rasterio.open(path +'prec_avr.tif')

land_cover = rasterio.open(path +'land_cover_raster.tif')
land_cover = land_cover.read(1)
geo_faults = rasterio.open(path +'geo_faults.tif')
roads = rasterio.open(path +'roads.tif')
rivers = rasterio.open(path +'rivers.tif')



target = rasterio.open(path +'train_raster.tif')

feature_list = [elevation, twi, slope_rad, aspect, curvature, land_cover]
                #rain_90, rain_avr, land_cover, geo_faults, roads, rivers]
add_feat = [rain_90, rain_avr, geo_faults, roads, rivers]
#reproduce
random_seed = 42
torch.manual_seed(random_seed)
np.random.seed(random_seed)

### create stats

In [ ]:
#RUN THIS ONCE TO CREATE DF

"""
how the stats are calculated
el = elevation.read(1)
mask = (el != 0) #filter out all cells with no data, approx lowest point in region is above 0
el = None

columns = ['elevation', 'twi', 'slope_rad', 'aspect', 'curvature', 'rain_90', 'rain_avr',
           'land_cover', 'geo_faults', 'roads', 'rivers']

index = ['mean', 'std']
stats_df = pd.DataFrame(index = index, columns = columns)

for i in range(len(feature_list)):
    col = columns[i]

    layer = feature_list[i].read(1)
    filtered_layer = layer[mask]
    mean = filtered_layer.mean()
    std = filtered_layer.std()

    stats_df[col] = [mean, std]



stats_df.to_csv(r'datasets/stats.csv')

"""

stats_df = pd.read_csv(path_drive +'stats.csv')

In [ ]:
window_Extracter = extractWindows(feature_readers=feature_list,
                                  target_reader=target, add_feat = add_feat)

In [ ]:
class MapData(Dataset):
    def __init__(self, window_extracter, rows, cols):
        self.extracter = window_extracter
        self.rows = rows
        self.cols = cols


    def __getitem__(self, index):
        row = self.rows[index]
        col = self.cols[index]

        X, X2 = self.extracter.__getFeatures__(row, col)
        Y = self.extracter.__getTarget__(row, col)


        X = torch.tensor(X, dtype=torch.float32)
        X2 = torch.tensor(X2,dtype=torch.float32 )
        Y = torch.tensor(Y, dtype=torch.float32)

        return X, X2, Y[0]


    def __len__(self):
        return len(self.rows) #same amount of data points



In [ ]:
train_row_cols = pd.read_csv(path_drive + 'row_col_target.csv')
#train_row_cols = train_row_cols.sample(frac=0.3, random_state = 42)

#count entries
counts  = train_row_cols['target'].to_frame().value_counts()
smallest_size = counts.iloc[-1]


# original subsets for each class
subsets = []
for prob in range(2):
    filtered = train_row_cols[train_row_cols['target'] == prob]
    subsets.append(filtered)

train_row_cols = None
# under-sample each subset
undersampled_subset = []
for i in subsets:
    shuffled_i = i.sample(frac = 1) #so data is randomly shuffles
    i = shuffled_i.iloc[:smallest_size].copy()
    undersampled_subset.append(i)

# concatenate the seven under-sampled subsets, and shuffle the data
undersampled_df = pd.concat(undersampled_subset)
undersampled_df = undersampled_df.sample(frac=1)


rows = undersampled_df['row'].values
cols = undersampled_df['col'].values

undersampled_df = None


mean = stats_df.iloc[0].values[1:].astype('float32')
std = stats_df.iloc[1].values[1:].astype('float32')

t_mean = np.concatenate((mean[:5], [mean[7]]))
t_std =  np.concatenate((std[:5], [std[7]]))

data_transform = transforms.Compose([
    transforms.Resize((100,100)),
    transforms.RandomHorizontalFlip(p=0.4),
    transforms.RandomVerticalFlip(p=0.25),
    transforms.Normalize(mean=t_mean, std =t_std)

])





In [ ]:
map = MapData(window_Extracter, rows, cols)

### Hyperparameters

In [ ]:
learning_rate = 0.0001
batch_size = 64
amount_train = 0.7
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")


In [ ]:
training_data_size = len(rows)
train_size = int(amount_train*training_data_size)
test_size = training_data_size - train_size

train_dataset, test_dataset = torch.utils.data.random_split(map, [train_size, test_size])

data_train = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
data_test = DataLoader(test_dataset, batch_size=batch_size, shuffle =False)

In [ ]:
class ConvNet(nn.Module):
    def __init__(self):
        super(ConvNet, self).__init__()
        self.conv1 = nn.Conv2d(6, 16, kernel_size=3, padding=1)
        self.pool = nn.MaxPool2d(2,2)
        self.conv2 = nn.Conv2d(16, 32, kernel_size=3, padding=1)

        self.fc1 = nn.Linear(25*25*32, 512)
        self.fc2 = nn.Linear(512, 32)
        self.fc3 = nn.Linear(32+5, 32)
        self.fc4 = nn.Linear(32, 1)


        self.dropout = nn.Dropout(p = 0.5)

    def forward(self, X, X2):

        X = self.pool(F.relu(self.conv1(X))) #11 x 100 x 100 input, out 16 x 50 x 50
        X = self.pool(F.relu(self.conv2(X))) # in: 16 x 50 x 50 , out 36 x 25 x 25


        X = X.view(-1, 25*25*32)
        X = F.relu(self.fc1(X))
        X = self.dropout(X)

        X = F.relu(self.fc2(X))
        X = self.dropout(X)

        X = torch.cat((X, X2.view(-1, 5)), dim=1)
        X = self.dropout(X)

        X = F.relu(self.fc3(X))

        X = torch.sigmoid(self.fc4(X))

        return X

In [ ]:
import torch.optim.lr_scheduler as lr_scheduler

CNN_DNN = ConvNet().to(device)
loss_fn = torch.nn.BCELoss()
loss_fn = loss_fn.to(device)
optimizer = torch.optim.Adam(CNN_DNN.parameters(), lr = learning_rate)
epochs = 3

scheduler = lr_scheduler.StepLR(optimizer, step_size=1, gamma=0.7)

In [ ]:
def train(epochs, CNN, loss_fn, optimizer,
          data_train, data_test, transformer, scheduler):
    print('Starting training..')
    for e in range(0, epochs):
        print('='*20)
        print(f'Starting epoch {e + 1}/{epochs}')
        print('='*20)

        train_loss = 0.
        val_loss = 0.

        CNN.train()
        # set model to training phase

        for train_step, (images,X2, labels) in enumerate(data_train):

            images = images.to(device)
            images = transformer(images)
            X2 = X2.to(device)
            labels = labels.to(device)
            # set the gradients to zero (you can do so by accessing the optimizer)
            optimizer.zero_grad()
            # compute outputs
            output = CNN(images, X2).to(device)


            # compute loss  (you have defined the loss_fn above!)
            loss = loss_fn(output, labels)

            #  use backward() so that the whole graph is differentiated w.r.t. the loss
            loss.backward()
            # performs a parameter update based on the current gradient (Note: you need to do it through the optimizer)
            optimizer.step()

            train_loss += loss.item()

            if train_step % 1000 == 0:

                if train_step % 2000 == 0 and train_step > 0:
                  scheduler.step()

                print('Evaluating at step', train_step)

                accuracy = 0

                CNN.eval()          # set model to eval phase

                for val_step, (images, X2, labels) in enumerate(data_test):
                    images = images.to(device)
                    images = transformer(images)
                    X2 = X2.to(device)
                    labels = labels.to(device)

                    outputs = CNN(images, X2).to(device)
                             # compute outputs

                    loss = loss_fn(outputs, labels)
                    loss = loss.to(device)
                        # compute loss
                    val_loss += loss.item()

                    preds = (outputs > 0.5).float()
                    accuracy += sum((preds.cpu() == labels.cpu()).numpy())

                    if val_step == 500:
                      break

                val_loss = val_loss
                val_loss /= (val_step + 1)
                accuracy = accuracy
                accuracy = accuracy/(val_step *64)
                print(f'Validation Loss: {val_loss:.4f}, Accuracy: {accuracy[0]:.4f}')

                CNN.train()



        train_loss /= (train_step + 1)

        print(f'Training Loss: {train_loss:.4f}')
    print('Training complete..')

In [ ]:
train(epochs, CNN_DNN, loss_fn=loss_fn, optimizer=optimizer,
      data_train=data_train, data_test=data_test,
      transformer= data_transform, scheduler = scheduler)

#stopped at step21000

Starting training..
Starting epoch 1/3


/usr/local/lib/python3.10/dist-packages/torchvision/transforms/functional.py:1603: UserWarning: The default value of the antialias parameter of all the resizing transforms (Resize(), RandomResizedCrop(), etc.) will change from None to True in v0.17, in order to be consistent across the PIL and Tensor backends. To suppress this warning, directly pass antialias=True (recommended, future default), antialias=None (current default, which means False for Tensors and True for PIL), or antialias=False (only works on Tensors - PIL will still use antialiasing). This also applies if you are using the inference transforms from the models weights: update the call to weights.transforms(antialias=True).
  warnings.warn(


Evaluating at step 0
Validation Loss: 0.6926, Accuracy: 0.4963
Evaluating at step 1000
Validation Loss: 0.3914, Accuracy: 0.8310
Evaluating at step 2000
Validation Loss: 0.3678, Accuracy: 0.8415
Evaluating at step 3000
Validation Loss: 0.3438, Accuracy: 0.8563
Evaluating at step 4000
Validation Loss: 0.3494, Accuracy: 0.8546
Evaluating at step 5000
Validation Loss: 0.3474, Accuracy: 0.8595
Evaluating at step 6000
Validation Loss: 0.3482, Accuracy: 0.8602
Evaluating at step 7000
Validation Loss: 0.3564, Accuracy: 0.8610
Evaluating at step 8000
Validation Loss: 0.3464, Accuracy: 0.8614
Evaluating at step 9000
Validation Loss: 0.3530, Accuracy: 0.8640
Evaluating at step 10000
Validation Loss: 0.3570, Accuracy: 0.8645
Evaluating at step 11000
Validation Loss: 0.3554, Accuracy: 0.8648
Evaluating at step 12000
Validation Loss: 0.3493, Accuracy: 0.8661
Evaluating at step 13000
Validation Loss: 0.3493, Accuracy: 0.8677
Evaluating at step 14000
Validation Loss: 0.3519, Accuracy: 0.8656
Evaluati

KeyboardInterrupt: ignored

In [ ]:

opt = torch.optim.Adam(CNN_DNN.parameters(), lr = learning_rate/10)
scheduler_2 = lr_scheduler.StepLR(opt, step_size=1, gamma=0.2)

train(epochs, CNN_DNN, loss_fn=loss_fn, optimizer=opt,
      data_train=data_train, data_test=data_test,
      transformer= data_transform, scheduler = scheduler_2)


#stop at step 7000

Starting training..
Starting epoch 1/3
Evaluating at step 0
Validation Loss: 0.3001, Accuracy: 0.8727
Evaluating at step 1000
Validation Loss: 0.2942, Accuracy: 0.8759
Evaluating at step 2000
Validation Loss: 0.2931, Accuracy: 0.8725
Evaluating at step 3000
Validation Loss: 0.2903, Accuracy: 0.8777
Evaluating at step 4000
Validation Loss: 0.2884, Accuracy: 0.8758
Evaluating at step 5000
Validation Loss: 0.2888, Accuracy: 0.8771
Evaluating at step 6000
Validation Loss: 0.2885, Accuracy: 0.8767
Evaluating at step 7000


KeyboardInterrupt: ignored

In [ ]:

torch.save(CNN_DNN, path_drive + 'cnn_dnn_6.pt')
torch.save(CNN_DNN.state_dict(), path_drive + 'cnn_dnn_dict_6.pt')

Test


In [ ]:
import geopandas as gpd
test_gpd = gpd.read_file(path_drive + 'Test.gpkg')
sample_sub = pd.read_csv(path_drive + 'Test.csv')

In [ ]:
rows_val = []
cols_val = []

val_transform = transforms.Compose([
    transforms.Resize((100,100)),
    transforms.Normalize(mean=t_mean, std =t_std)
])

for id, geom in zip(sample_sub['ID'], test_gpd['geometry']):
  #geom = test_gpd[test_gpd['ID'] == id]
  #geom = geom['geometry'].values[0]
  x, y = geom.xy[0][0], geom.xy[1][0]
  r, c= roads.index(x,y)

  rows_val.append(r)

  cols_val.append(c)


val_data = MapData(window_Extracter, rows_val, cols_val)

val_loader = DataLoader(val_data, batch_size=batch_size, shuffle =False)

target = np.array([])

CNN_DNN.eval()

for val_step, (images, X2, labels) in enumerate(val_data):
                    images = images.to(device)
                    images = val_transform(images)
                    labels = labels.to(device)
                    X2 = X2.to(device)

                    outputs = CNN_DNN(images, X2).to(device)
                             # compute outputs
                    pred = (outputs > 0.5).float()

                    t = pred.cpu()

                    target = np.append(target, t)

print(len(target))

40000


In [ ]:
sample_sub = pd.read_csv(path_drive + 'Test.csv')

sample_sub['Target'] = target.astype(np.int8)

sample_sub.to_csv(path_drive + 'submission_cnn_6.csv', index=False)